# Run

In [ ]:
!pip install diffusers
!pip uninstall jax jaxlib -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 37.6 MB/s eta 0:00:00
Found existing installation: jax 0.4.33
Uninstalling jax-0.4.33:
  Successfully uninstalled jax-0.4.33
Found existing installation: jaxlib 0.4.33
Uninstalling jaxlib-0.4.33:
  Successfully uninstalled jaxlib-0.4.33


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Much better vae module
# Load VAE model
from diffusers import AutoencoderKL
device = "cuda:0"
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse").to(device)
vae.eval()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
        )
        (downsamplers): ModuleList(
          (0): Downsample2D(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
          )
        )
      )
      (1): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (c

# Residual UNET

In [ ]:
# Install necessary libraries (if not already installed)
!pip install ftfy pycocotools diffusers optuna

# Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision.models import vgg19
from PIL import Image
from tqdm import tqdm
import os
import numpy as np
import logging
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from diffusers import AutoencoderKL
import random
import zipfile
import requests
from pycocotools.coco import COCO
from concurrent.futures import ThreadPoolExecutor, as_completed

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
model_path = "Models/PixelLevelRestorationVAECont2.3"

# Set up logging
log_file = f'/content/drive/MyDrive/{model_path}/training_log.txt'
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(
    filename=log_file,
    filemode='w',
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.INFO
)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load VAE Model
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse").to(device)
vae.eval()
for param in vae.parameters():
    param.requires_grad = False

# Download COCO Training Images
data_dir = '/content/coco_data'
annotations_dir = '/content/coco_annotations'
os.makedirs(data_dir, exist_ok=True)
os.makedirs(annotations_dir, exist_ok=True)

# Download COCO annotations
annotations_url = 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip'
annotations_zip = 'annotations_trainval2017.zip'

if not os.path.exists(os.path.join(annotations_dir,'instances_train2017.json')):
    print('Downloading COCO annotations...')
    response = requests.get(annotations_url, stream=True)
    with open(annotations_zip, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    print('Extracting COCO annotations...')
    with zipfile.ZipFile(annotations_zip, 'r') as zip_ref:
        zip_ref.extractall(annotations_dir)
    os.remove(annotations_zip)
else:
    print('COCO annotations already downloaded.')

# Function to download a single image
def download_image(img_data, save_dir):
    img_url = img_data['coco_url']
    img_name = os.path.join(save_dir, img_data['file_name'])

    if not os.path.exists(img_name):
        try:
            response = requests.get(img_url, stream=True)
            if response.status_code == 200:
                with open(img_name, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
                logging.info(f"Downloaded {img_name}")
            else:
                logging.error(f"Failed to download {img_name}: HTTP {response.status_code}")
        except Exception as e:
            logging.error(f"Failed to download {img_name}: {e}")
    else:
        logging.info(f"Image {img_name} already exists, skipping download.")

# Function to download images using parallel downloading
def download_random_coco_images(num_images=10000, save_dir='/content/coco_data'):
    coco = COCO('/content/coco_annotations/annotations/instances_train2017.json')
    image_ids = coco.getImgIds()
    random_ids = random.sample(image_ids, num_images)

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        for img_id in random_ids:
            img_data = coco.loadImgs(img_id)[0]
            futures.append(executor.submit(download_image, img_data, save_dir))

        for future in as_completed(futures):
            future.result()

# Download images if not already done
if not os.listdir(data_dir):
    logging.info("Downloading COCO images...")
    download_random_coco_images(num_images=10000, save_dir=data_dir)
else:
    logging.info("COCO images already downloaded.")

# Quantize functions (operate in latent space)
def quantize_batch(latents, parameters):
    parameters = parameters.view(-1, 1, 1, 1)
    quantized_latents = (latents / (255 * parameters) + 0.5).clamp(0, 1)
    quantized = (quantized_latents * 255.0 + 0.5).clamp(0, 255).type(torch.uint8)
    quantized = quantized.float() / 127.5 - 1
    return quantized

def unquantize_batch(quantized, parameters):
    parameters = parameters.view(-1, 1, 1, 1)
    unquantized = (quantized + 1) * 127.5
    unquantized = unquantized / 255.0
    unquantized_latents = (unquantized - 0.5) * (255 * parameters)
    return unquantized_latents.to(quantized.device)

# Custom Dataset Class with Smart Cropping
class VAEDataset(Dataset):
    def __init__(self, image_dir, parameters, crop_size=256, transform=None):
        self.image_paths = sorted([os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(('.jpg', '.png'))])
        self.parameters = parameters
        self.crop_size = crop_size
        self.transform = transform

    def __len__(self):
        return len(self.image_paths) * len(self.parameters)

    def __getitem__(self, idx):
        img_idx = idx // len(self.parameters)
        param_idx = idx % len(self.parameters)
        parameter = self.parameters[param_idx]
        parameter = torch.tensor([parameter], dtype=torch.float32)

        image_path = self.image_paths[img_idx]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        i, j, h, w = transforms.RandomCrop.get_params(image, output_size=(self.crop_size, self.crop_size))
        image = image[:, i:i+h, j:j+w]

        return image, parameter
class ConditionalResize:
    def __init__(self, min_size):
        self.min_size = min_size

    def __call__(self, img):
        w, h = img.size
        if min(w, h) < self.min_size:
            img = transforms.Resize(self.min_size)(img)
        return img
# Data Transformation with Enhanced Augmentation
transform = transforms.Compose([
    ConditionalResize(144),
    transforms.RandomResizedCrop(128, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.02),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Paths to images
image_dir = '/content/coco_data'

# List of parameter values with variability
parameters = [0.58215, 0.78215, 0.98215]

# Dataset and DataLoader
crop_size = 128
dataset = VAEDataset(image_dir, parameters, crop_size=crop_size, transform=transform)

# Use a subset of data if needed
subset_size = 10000
dataset = torch.utils.data.Subset(dataset, range(subset_size))

# Split Dataset into Training and Validation Sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Adjusted batch size for memory constraints
batch_size = 32  # Reduced batch size due to increased memory usage
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Define Self-Attention Block
class SelfAttention(nn.Module):
    def __init__(self, in_dim):
        super(SelfAttention, self).__init__()
        self.in_dim = in_dim
        self.query_conv = nn.Conv2d(in_dim, in_dim // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_dim, in_dim // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_dim, in_dim, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        batch, C, width, height = x.size()
        proj_query = self.query_conv(x).view(batch, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch, -1, width * height)
        energy = torch.bmm(proj_query, proj_key)
        attention = self.softmax(energy)
        proj_value = self.value_conv(x).view(batch, -1, width * height)

        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        out = out.view(batch, C, width, height)

        out = self.gamma * out + x
        return out

# Define U-Net with Attention and Parameter Embedding
class UNetWithParams(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):  # Adjusted channels for images
        super(UNetWithParams, self).__init__()
        self.in_channels = in_channels

        self.param_fc = nn.Linear(1, 16)

        self.enc1 = self.conv_block(in_channels + 16, 64)
        self.enc2 = self.conv_block(64, 128)
        self.att1 = SelfAttention(128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)

        self.bottleneck = self.conv_block(512, 1024)

        self.dec4 = self.up_conv(1024, 512)
        self.dec4_conv = self.conv_block(1024, 512)

        self.dec3 = self.up_conv(512, 256)
        self.dec3_conv = self.conv_block(512, 256)
        self.att2 = SelfAttention(256)

        self.dec2 = self.up_conv(256, 128)
        self.dec2_conv = self.conv_block(256, 128)

        self.dec1 = self.up_conv(128, 64)
        self.dec1_conv = self.conv_block(128, 64)

        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        return block

    def up_conv(self, in_channels, out_channels):
        up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        return up

    def forward(self, x, params):
        # Embed parameters
        param_embedding = self.param_fc(params)
        param_embedding = param_embedding.view(-1, 16, 1, 1)
        param_embedding = param_embedding.expand(-1, -1, x.size(2), x.size(3))
        x = torch.cat((x, param_embedding), dim=1)

        # Encoder
        e1 = self.enc1(x)
        e2 = self.enc2(F.max_pool2d(e1, 2))
        e2 = self.att1(e2)
        e3 = self.enc3(F.max_pool2d(e2, 2))
        e4 = self.enc4(F.max_pool2d(e3, 2))

        # Bottleneck
        b = self.bottleneck(F.max_pool2d(e4, 2))

        # Decoder
        d4 = self.dec4(b)
        d4 = torch.cat((d4, e4), dim=1)
        d4 = self.dec4_conv(d4)

        d3 = self.dec3(d4)
        d3 = torch.cat((d3, e3), dim=1)
        d3 = self.dec3_conv(d3)
        d3 = self.att2(d3)

        d2 = self.dec2(d3)
        d2 = torch.cat((d2, e2), dim=1)
        d2 = self.dec2_conv(d2)

        d1 = self.dec1(d2)
        d1 = torch.cat((d1, e1), dim=1)
        d1 = self.dec1_conv(d1)

        out = self.final_conv(d1)
        return out

# Initialize model
model = UNetWithParams().to(device)

# Loss Functions
criterion_mse = nn.MSELoss()

# Perceptual Loss (VGG19)
vgg = vgg19(pretrained=True).features[:12].to(device).eval()
for param in vgg.parameters():
    param.requires_grad = False

def perceptual_loss(output, target):
    output_features = vgg(output)
    target_features = vgg(target)
    return F.l1_loss(output_features, target_features)

# Multi-Scale Loss
def multi_scale_loss(output, target):
    loss = 0
    for scale in [1, 0.5, 0.25]:
        scaled_output = F.interpolate(output, scale_factor=scale, mode='bilinear', align_corners=False)
        scaled_target = F.interpolate(target, scale_factor=scale, mode='bilinear', align_corners=False)
        loss += F.mse_loss(scaled_output, scaled_target)
    return loss

# Optimizer and Scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)


# Path to your saved checkpoint
checkpoint_path = '/content/drive/MyDrive/Models/PixelLevelRestorationVAECont2.2/unet_best_checkpoint.pth'

# Load the saved checkpoint
checkpoint = torch.load(checkpoint_path)

# Load model state
model.load_state_dict(checkpoint['model_state_dict'])

# Scalars for mixed precision
scaler = GradScaler()

# Training Loop
num_epochs = 50
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, parameters in tqdm(train_loader):
        images = images.to(device)
        parameters = parameters.to(device)

        optimizer.zero_grad()

        with autocast():
            # Encode images using VAE
            with torch.no_grad():
                vae_images = (vae.encode(images).latent_dist.mode() * 0.18215).float()

            # Quantize and unquantize latents
            quantized_latents = quantize_batch(vae_images, parameters)
            unquantized_latents = unquantize_batch(quantized_latents, parameters)

            # Decode the unquantized latents to images
            with torch.no_grad():
                decoded_unquantized_images = vae.decode(unquantized_latents / 0.18215).sample

            # Compute residuals in pixel space
            residuals = images - decoded_unquantized_images

            # Model prediction
            outputs = model(decoded_unquantized_images, parameters)

            # Reconstructed images
            reconstructed_images = outputs + decoded_unquantized_images

            # Normalize the images for VGG
            def preprocess_for_vgg(img):
                img = (img + 1) / 2  # Scale from [-1,1] to [0,1]
                imagenet_mean = torch.tensor([0.485, 0.456, 0.406]).view(1,3,1,1).to(device)
                imagenet_std = torch.tensor([0.229, 0.224, 0.225]).view(1,3,1,1).to(device)
                img = (img - imagenet_mean) / imagenet_std
                return img

            output_vgg_input = preprocess_for_vgg(reconstructed_images)
            target_vgg_input = preprocess_for_vgg(images)

            # Compute losses
            mse_loss = criterion_mse(outputs, residuals)
            perc_loss = perceptual_loss(output_vgg_input, target_vgg_input)
            ms_loss = multi_scale_loss(output_vgg_input, target_vgg_input)
            total_loss = mse_loss + 0.01 * perc_loss + 0.1 * ms_loss

        scaler.scale(total_loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += total_loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.6f}')

    # Validation Loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, parameters in val_loader:
            images = images.to(device)
            parameters = parameters.to(device)

            with autocast():
                # Encode images using VAE
                vae_images = (vae.encode(images).latent_dist.mode() * 0.18215).float()

                # Quantize and unquantize latents
                quantized_latents = quantize_batch(vae_images, parameters)
                unquantized_latents = unquantize_batch(quantized_latents, parameters)

                # Decode the unquantized latents to images
                decoded_unquantized_images = vae.decode(unquantized_latents / 0.18215).sample

                # Compute residuals in pixel space
                residuals = images - decoded_unquantized_images

                # Model prediction
                outputs = model(decoded_unquantized_images, parameters)

                # Reconstructed images
                reconstructed_images = outputs + decoded_unquantized_images

                # Normalize the images for VGG
                output_vgg_input = preprocess_for_vgg(reconstructed_images)
                target_vgg_input = preprocess_for_vgg(images)

                # Compute losses
                mse_loss = criterion_mse(outputs, residuals)
                perc_loss = perceptual_loss(output_vgg_input, target_vgg_input)
                ms_loss = multi_scale_loss(output_vgg_input, target_vgg_input)
                total_loss = mse_loss + 0.01 * perc_loss + 0.1 * ms_loss

            val_loss += total_loss.item() * images.size(0)

    val_loss /= len(val_dataset)
    scheduler.step(val_loss)
    logging.info(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.6f}, Validation Loss: {val_loss:.6f}')
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.6f}, Validation Loss: {val_loss:.6f}')

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'training_loss': epoch_loss,
            'validation_loss': val_loss,
        }
        torch.save(checkpoint, f'/content/drive/MyDrive/{model_path}/unet_best_checkpoint.pth')
        logging.info(f'Best model saved at epoch {epoch+1}')
        print(f'Best model saved at epoch {epoch+1}')
    checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'training_loss': epoch_loss,
            'validation_loss': val_loss,
        }
    torch.save(checkpoint, f'/content/drive/MyDrive/{model_path}/unet_checkpoint_epoch_{epoch+1}.pth')
    logging.info(f'Model checkpoint saved at epoch {epoch+1}')
    # Save the model checkpoint periodically
    if (epoch + 1) % 5 == 0:
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'training_loss': epoch_loss,
            'validation_loss': val_loss,
        }
        torch.save(checkpoint, f'/content/drive/MyDrive/{model_path}/unet_checkpoint_epoch_{epoch+1}.pth')
        logging.info(f'Model checkpoint saved at epoch {epoch+1}')

# Save the final model
checkpoint = {
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'training_loss': epoch_loss,
    'validation_loss': val_loss,
}
torch.save(checkpoint, f'/content/drive/MyDrive/{model_path}/unet_final_checkpoint.pth')
logging.info('Final model saved.')

# Release GPU memory
from google.colab import runtime
runtime.unassign()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.0 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extracting COCO annotations...
loading annotations into memory...
Done (t=18.67s)
creating index...
index created!


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also u

Epoch [1/50], Training Loss: 0.082638



<ipython-input-4-3456f19bdb63>:422: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch [1/50], Training Loss: 0.082638, Validation Loss: 0.083205
Best model saved at epoch 1


...

Epoch [49/50], Training Loss: 0.080378


Epoch [49/50], Training Loss: 0.080378, Validation Loss: 0.081203


100%|██████████| 282/282 [05:33<00:00,  1.18s/it]

Epoch [50/50], Training Loss: 0.080383


Epoch [50/50], Training Loss: 0.080383, Validation Loss: 0.081530


In [ ]:
import os

# Directory where the images are saved
data_dir = '/content/coco_data'

# Count the number of files in the directory
num_images = len([name for name in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, name))])

print(f"Total number of images downloaded: {num_images}")

# Check if the count matches the expected number
if num_images == 10000:
    print("Successfully downloaded all 10000 images.")
else:
    print(f"Only {num_images} images downloaded, expected 10000.")
